<a href="https://colab.research.google.com/github/antonum/Timescale-Workshops/blob/main/Terraform_Timescale_Cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Provision Timescale service with Terraform

### References

- Timescale terraform provider https://registry.terraform.io/providers/timescale/timescale/latest/docs
- Timescale documentation https://docs.timescale.com/use-timescale/latest/integrations/terraform/

### Prerequisites

You should have an existing project in the Terraform cloud. You can create a free 30 days trial (no credit card required) here: https://console.cloud.timescale.com/

In [ ]:
%%bash
# install terraform
wget -O - https://apt.releases.hashicorp.com/gpg | sudo gpg --dearmor -o /usr/share/keyrings/hashicorp-archive-keyring.gpg
echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/hashicorp-archive-keyring.gpg] https://apt.releases.hashicorp.com $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/hashicorp.list
sudo apt update && sudo apt install terraform -q

## Create .tf files

Following [Terraform best practices](https://www.terraform-best-practices.com/code-structure#getting-started-with-the-structuring-of-terraform-configurations) we'll structure code as `main.tf`, `variables.tf` and `outputs.tf` and `terraform.tfvars`. The code below creates these files in the home directory of this notebook.

Instead of putting your credentials in `terraform.tfvars` you can use `-var` argment in terraform command line or environment variables:

```
export TF_VAR_ts_access_key=01JR8823FVNHNQZY0TJBXXXXX
export TF_VAR_ts_secret_key=j68Od78cDOKJItUCnWoC7PQPkGSkIpbPaaU2aquwkErrqIYgB3xCmAxxxXXXxx
export TF_VAR_ts_project_id=ocssxxxx
```


In [ ]:
%%bash
# Create terraform.tfvars
tee terraform.tfvars <<'EOF'
# use keys and project ID from your existing project

ts_access_key="01JR8823FVNHNQZY0TJBXXXX"
ts_secret_key="j68Od78cDOKJItUCnWoC7PQPkGSkIpbPaaU2aquwkErrqIYgB3xCmAQXV71xxxxx"
ts_project_id="ocssgixxxx"

EOF

In [ ]:
%%bash
# Create main.tf
tee main.tf <<EOF
terraform {
  required_providers {
    timescale = {
      source  = "timescale/timescale"
      version = "~> 1.13.1"
    }
  }
}

# Authenticate using client credentials.
# They are issued through the Timescale UI.
# When required, they will exchange for a short-lived JWT to do the calls.
provider "timescale" {
  project_id = var.ts_project_id
  access_key = var.ts_access_key
  secret_key = var.ts_secret_key
}

resource "timescale_service" "test" {
  name       = "anton-tf-test"
  milli_cpu  = 500
  memory_gb  = 2
  region_code = "us-east-1"
  enable_ha_replica = false
  timeouts = {
    create = "30m"
  }
}
EOF



In [ ]:
%%bash
# Create variables.tf
tee variables.tf <<EOF
variable "ts_project_id" {
  type = string
  //default = "ocssgijxxx"
}

variable "ts_access_key" {
  type = string
  //default = "01JR8823FVNHNQZY0TJB6XXXXX"
}

variable "ts_secret_key" {
  type      = string
  sensitive = true
  //default = "j68Od78cDOKJItUCnWoC7PQPkGSkIpbPaaU2aquwkErrqIYgB3xCmxxXXxxXX"
}
EOF

In [ ]:
%%bash
# Create outputs.tf
tee outputs.tf <<'EOF'
output "timescale_service_hostname" {
  value = timescale_service.test.hostname
}

output "timescale_service_port" {
  value = timescale_service.test.port
}

output "timescale_service_password" {
  value = timescale_service.test.password
  sensitive = true
}

output "timescale_service_psql" {
  value = "psql -d \"postgres://tsdbadmin:${timescale_service.test.password}@${timescale_service.test.hostname}:${timescale_service.test.port}/tsdb?sslmode=require\""
  sensitive = true
}
EOF

In [ ]:
%%bash
terraform init

In [ ]:
%%bash
terraform validate

In [ ]:
%%bash
terraform plan

## Provision Timescale Cloud service

In the cell below enter access key, secret key and project ID of your timescale cloud project. To get the key navigate to Project (upper left corner) -> Project Settings -> Client Credentials -> Create Credentials

**WARNING!!!** provisioning new service on Timescale Cloud might incur additional costs. Make sure to delete the service. Cleanup cell below.

In [ ]:
%%bash
terraform apply  -auto-approve

In [ ]:
# output psql connection string for your service
!terraform output timescale_service_psql

## Cleanup

In [ ]:
%%bash
# delete service
terraform destroy  -auto-approve